### Now we have both our final datasets and we have to start the pre-processing phase
The two datasets are a bit unbalanced, for this reason we should collect more data for the control one. Let's see later how to do that. 

In [ ]:
#import required libraries 
import csv
import json
import os
import requests  
import time
import datetime
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
import nltk
import string
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from zipfile import ZipFile
from nltk import sent_tokenize
from nltk import word_tokenize

In [ ]:
#open experimental group dataset
experimental_df_1 = pd.read_csv('/content/drive/MyDrive/Computational_Linguistics_Project /datasets/Cleaned_Depression_Vs_Suicide.csv')
experimental_df_1 = experimental_df_1.reset_index(drop=True)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (0,1) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#dropping duplicates
experimental_df_1 = experimental_df_1.drop_duplicates(subset=['text'])
experimental_df_1 = experimental_df_1.reset_index(drop=True)

In [ ]:
#Dropping rows with NaN values
experimental_df_1.dropna(axis = 0, inplace = True)
len(experimental_df_1)

609770

In [ ]:
cleaned_intero_tozip = experimental_df_1.copy()

NB: the posts ***non-suicide*** and ***depression***, are the same  come from the same subreddit channel. You can easily filter for them in order to obtain the single word embedding models. 

In [ ]:
#open control group dataset 
control_df_1 = pd.read_csv('/content/drive/MyDrive/Computational_Linguistics_Project /datasets/df_control_group.csv', index_col= 0)

In [ ]:
#remove duplicates 
control_df_1 = control_df_1.drop_duplicates(subset=['text'])
control_df_1 = control_df_1.reset_index(drop=True)
control_df_1 = control_df_1.dropna()


In [ ]:
len(control_df_1)

12058

## CLEANING TEXT  of both datasets

In [ ]:
#list of stopwords that we do not want to be removed because they are interesting for our analysis
do_not_remove = ['do','me', 'my', 'i', 'myself', 'we', 'our', 'you', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'her', 'hers', 'herself', 'them', 'their', 'themselves', 'up', 'down', 'ourselves', 'ours']

In [ ]:
nltk.download('stopwords')
stop_words = stopwords.words("english")
#extend list of stopwords according to an arbitrary decision that I made
stop_words.extend(['it', 'is'])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
for word in do_not_remove:
    stop_words.remove(word)

### Here I defined the function which contains all the elements for cleaning the Reddit text.

In [ ]:
import re

def clean_up(testo):
    punctu = '"#$%&\'()*+-/:;<=>@[\\]^_`{|}~'
    testo = str(testo)
    testo = testo.lower()
    testo = re.sub(r"(i'm)", 'i am', testo)
    testo = re.sub(r'ama', 'i am a', testo)
    testo = re.sub(r'iama', 'i am a', testo)
    testo = re.sub(r"(i've)", 'i have', testo)
    testo = re.sub(r"i'm", 'i am', testo )
    testo = re.sub(r"i've", 'i have', testo)
    testo = re.sub(r"haven't'", 'have not', testo)
    testo = re.sub(r'(?<=[.,?])(?=[^\s])', r' ', testo) 
    testo = ' '.join([word for word in testo.split(' ') if word not in stop_words]) #remove stopwords
    testo = re.sub(r'\'\w+', '', testo)  #remove ticks and the next character
    testo = testo.encode('ascii', 'ignore').decode()
    testo = re.sub(r'https*\S+', ' ', testo) #remove hyperlinks
    testo = re.sub(r'@\S+', ' ', testo) #remove mentions
    testo = re.sub(r'#\S+', ' ', testo) #remove hastags
    testo = re.sub('[%s]' % re.escape(punctu), ' ', testo) #remove punctuation (except .,?! that we will need to sent tokenize later)
    testo = re.sub(r'\w*\d+\w*', '', testo) #remove numbers
    testo = re.sub(r'\s{2,}', ' ', testo)   #remove extra spaces
    return testo
    

In [ ]:
cleaned_intero_tozip['cleaned_text'] = cleaned_intero_tozip.text.apply(clean_up)

In [ ]:
print(cleaned_intero_tozip.iloc[40]['cleaned_text'])
#clean_up(cleaned_intero_tozip.iloc[10]['text']))

mood changes within space couple minutes normal? ? also, pick me up suggestions like music, activities etc help me up? hi guys, basically i emotional wreck. diagnosed severe derpression my first post regarding matter. suffering years sought help things took turn worst.
i know supportive generally reddit could really use pick me up my virtual peers nothing else.
i one question thats really bugging me is, normal mood deteriorate withing space minutes trigger point hit even without it?
its really bugging me i thought id ask. tbh help pic me ups welcome, shit really ruining aspects my personal life making seem worse
cheers guys


In [ ]:
control_df_1['cleaned_text'] = control_df_1.text.apply(clean_up)

In [ ]:
#print(control_df_1.iloc[5678]['text'])
#control_df_1.iloc[5678]['cleaned_text']

'hey! i one two co founders ideal, personal development strategy platform, eventual full fledged clothing brand! i experienced many hardships life offer. i born abusi household, branded social outcast school, depressed many years. however, my determination, sown my mind inspirational words my mentors, allowed me persevere my struggles. i challenged my inner demons, slowly conquering them one by one my daily quest self iprove. several years intense personal growth, i began college. i succeeded socially, morphing one men i aspired my younger years. nurturing potential others, watching them become something believed ipossible, my life long passion. month ago, i withdrew unirsity maryland assist you journey become ideal. feel free ask me anything you like! proof com a my story youtube. com watch? v amp t our website www. becomeideal. com\nsocial media contact us info com'

#### Here I have built this function in order to rescue the ***I*** which were masked and attcahed to the verb.

In [ ]:
def rescue_i(testo):
    testo = re.sub(r'im', 'i', testo)
    testo = re.sub(r'ive', 'i', testo)
    
    return testo

In [ ]:
#apply the function to both datasets 
control_df_1['cleaned_text'] = control_df_1.cleaned_text.apply(rescue_i)
cleaned_intero_tozip['cleaned_text'] = cleaned_intero_tozip.text.apply(rescue_i)

In [ ]:
#print(control_df_1.iloc[5675]['cleaned_text'])
#print(cleaned_intero_tozip.iloc[5677]['cleaned_text'])

#### Here below I have implemented the function which perform the tokenization
It's built thanks to other two function, a word tokenizer and a sentence tokenizer.
Moreover, I defined other two functions, one able to save my tokenized posts into a ***zip_file***, the other able to unzip the stored file and retrieve the tokenized posts. 

In [ ]:
def sent_tokenizer(text):
    return re.findall(r".*?[.!\?]",text)

#_____________________________________________________________

def word_tokenizer(sentence):
    punct = r"""([A-z])([,;:\?!\."'])"""
    temp_sentence =  re.sub(punct, r"\1 \2", sentence)
    toks = temp_sentence.split()
    temp_out =[]
    # splitting english possessive
    for tok in toks:
        if re.search(r"([A-z]+)’s?$", tok):
            temp_out.extend(re.sub(r"([A-z]+)(’s?)$", r"\1 \2", tok).split())
        else:
            temp_out.append(tok)
    return temp_out 

#_____________________________________________________________

def my_tokenizer(text):
    import string 
    punct = string.punctuation 
    sentences = sent_tokenizer(text)
    tokenized_text = []
    for sent in sentences:
        if len(sent) > 1:
            tokens = word_tokenizer(sent)
            tokens_1 = [tok for tok in tokens if not tok in punct]
            tokenized_text.append(tokens_1)
    return tokenized_text


In [ ]:
def text2zip(path, lista_posts):
    
    arch_out = ZipFile(path, "w")
    out = []
    for i,article in enumerate(lista_posts):
        tok_sent = my_tokenizer(article)
        if len(tok_sent) > 2 and len(tok_sent[0]) > 1:
            
            out.append(tok_sent)
            text_article = "\n".join([" * ".join(tokens) for tokens in tok_sent]) # dividing tokens with " * "
            #print(text_article)
            arch_out.writestr(f"{i}_{tok_sent[0][0]}.txt", text_article)
    arch_out.close()
    return out

#_____________________________________________________________

def zip2text(path): 
    arch_in = ZipFile(mypath, "r")
    files_name = arch_in.namelist()
    out = {}
    for file in files_name:
        text = arch_in.read(file).decode('utf-8')
        article = [sent.split(" * ") for sent in text.split("\n")]
        out[file] = article
    arch_in.close()
    return out

In [ ]:
#trying to visualize the text once it has been cleaned; an asterisk has been put between each token just to have a better visualization
mental_illness = cleaned_intero_tozip['cleaned_text'][:10]
out = []
for i,article in enumerate(mental_illness):
    tok_sent = my_tokenizer(article)
    out.append(tok_sent)
    text_article = "\n".join([" * ".join(tokens) for tokens in tok_sent])
    print(text_article)

Feeling * a * bit * depressedI * 've * been * in * a * big * low * all * weekend
I * don * 't * know * why * death * has * actually * crossed * my * mind * a * few * ties
I * don * 't * care * to * admit * it * to * any * of * my * friend * because * I * feel * disconnected * from * everyone
I * feel * so * lonely * yet * I * feel * so * overwhelmed * that * I * can * 't * really * talk * to * anyone
I * 've * been * pretty * isolated * for * the * past * few * days * mostly * doing * work
I * just * want * someone * to * talk * to * it * doesn * 't * have * to * be * about * depression
I * 'm * a * male * college * student * about * to * graduate
Was * going * to * hang * myself * but * didn * 't * have * guts * enough * to * kick * away * the * chairI * was * all * set * to * hang * myself * this * morning
Had * the * noose * around * my * neck * was * standing * on * the * chair * the * whole * nine * yards
I * just * couldn * 't * get * the * guts * to * kick * away * the * chair
F

In [ ]:
control_group = control_df_1['cleaned_text'][:]
post_control = list(control_group)

In [ ]:
#I save the archive file with posts related to control group
mypath = "/content/drive/MyDrive/Computational_Linguistics_Project /myarchive_control.zip" 
lista_1 = text2zip(mypath,post_control)

In [ ]:
#just checking
lista_1[800][:]

[['i',
  'author',
  'two',
  'novels',
  'dead',
  'cats',
  'reflections',
  'parenthood',
  'anarchy',
  'lies'],
 ['i',
  'writing',
  'dystopian',
  'novels',
  'dystopia',
  'publishing',
  'world',
  'going',
  'end',
  'i',
  'thoughts'],
 ['check', 'my', 'books', 'short', 'stories', 'here', 'jessemckinnell'],
 ['com', 'proof', 'redd'],
 ['it']]

In [ ]:
mental_illness = cleaned_intero_tozip['cleaned_text'][:]
post_illness = list(mental_illness)

In [ ]:
#I save the archive file with posts related to mental illness
mypath_2 = "/content/drive/MyDrive/Computational_Linguistics_Project /myarchive_illness.zip" 
lista_2 = text2zip(mypath_2,post_illness)

In [ ]:
control_df_1.to_csv('FINALE_controllo.csv')

In [ ]:
experimental_df_1.to_csv('FINALE_sperimentale.csv')

### Here I split the experimental group into two 

In [ ]:
depression = experimental_df_1[experimental_df_1['class'] == 'depression']

In [ ]:
depression['cleaned_text'] = depression.text.apply(clean_up)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
depression.to_csv('depression_finale.csv')

In [ ]:
suicide = experimental_df_1[experimental_df_1['class'] == 'SuicideWatch']

In [ ]:
suicide['cleaned_text'] = suicide.text.apply(clean_up)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
suicide.to_csv('suicidio_finale.csv')